In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def get_t(labels_path):
    with open(labels_path) as f:
        training_labels = [int(x) for x in f.read().splitlines()]
    return np.array(training_labels)

In [ ]:
def get_T_matrix(t):
    # Labels matrix, each row representing one class, and the columns represent whether
    # the image belongs to that class (row index) or not (-1, 1) 
    T = np.zeros((10, len(t)))
    for i in range(len(T)):
        T[i] = np.where(t == i, 1, -1)
    return T

In [ ]:
def initialize_weights_matrix():
    # 10 weight vectors (classes), each one is all zeros except for the first feature.
    weight_matrix =  np.zeros((785, 10))
    weight_matrix[0] = np.ones(10)
    return weight_matrix.T

In [ ]:
def get_confusion_matrix(x_delta, weights_matrix, true_labels):    
    predictions = np.dot(x_delta, weights_matrix)
    conf_matrix = np.zeros((10, 10))
    for i, row in enumerate(predictions):
        # The index (class) of the maximum value (most probable class)
        prediction = np.where(row == row.max())[0][0]
        true_value = true_labels[i]
        conf_matrix[true_value][prediction] += 1

    return conf_matrix

In [ ]:
def get_x_delta(folder_name, number_images):
    x_input_points = np.zeros((0, 784))
    for i in range(1, number_images + 1): # +1 Since it's exclusive
        img_path = '{}/{}.jpg'.format(folder_name, i)
        x_input_points = np.append(x_input_points, plt.imread(img_path).reshape(1, 784), axis=0)

    return np.append(x_input_points, np.ones((x_input_points.shape[0], 1)), axis=1)

In [ ]:
# Loading in a separate cell to avoid multiple loads.
x_delta = get_x_delta('Train', 2400)
T = get_T_matrix(get_t('Train/Training Labels.txt'))

In [ ]:
# Train all learning rates for 500 epochs.
learning_rates_powers = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
resulting_weights = []

for e_p in learning_rates_powers: # Eta Power "e_p"
    w_m = initialize_weights_matrix()
    for c in range(0,10): # Class "c"
        for _ in range(0,500): # Iteration/Epoch
            for i in range(0, 2400): # Image "i"
                r = 1 if np.dot(w_m[c], x_delta[i]) >=0 else -1
                if r != T[:,i][c]:
                    # Update weight-vector if image "i" is misclassified
                    w_m[c] += (10 ** (-1 * e_p)) * (np.multiply(x_delta[i], T[:,i][c]))
    resulting_weights += [(e_p, w_m)]

In [ ]:
x_delta_test = get_x_delta('Test', 200)
test_true_labels = get_t('Test/Test Labels.txt')
for eta_power, weights_matrix in resulting_weights:
    confusion_matrix = get_confusion_matrix(x_delta_test, weights_matrix.T, test_true_labels)
    plt.imshow(confusion_matrix)
    plt.savefig('Confusion-{}.jpg'.format(eta_power))